# 01. BQuant Data Module Testing

Тестирование функций загрузки, обработки и валидации данных из модуля `bquant.data`.

## План тестирования:
1. **Настройка окружения** - импорты и установка директории данных
2. **Тестирование loader** - загрузка CSV файлов  
3. **Обзор доступных данных** - символы и таймфреймы

In [2]:
# Настройка окружения
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Добавляем путь к пакету bquant
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Project root: {project_root}")
print(f"🐍 Python path: {sys.path[0]}")
print(f"📊 Pandas version: {pd.__version__}")

📁 Project root: c:\Users\g\Documents\pro\bquant
🐍 Python path: c:\Users\g\Documents\pro\bquant
📊 Pandas version: 2.3.1


In [3]:
from bquant.core.logging_config import setup_logging

setup_logging(profile="research")

14:57:28 - bquant - INFO - Система логгирования BQuant инициализирована (консоль: INFO)
14:57:28 - bquant - INFO - Логи сохраняются в файл: c:\Users\g\Documents\pro\bquant\logs\bquant.log (уровень: INFO)


<Logger bquant (DEBUG)>

In [14]:
# Импорты из bquant
from bquant.core.config import (
    get_data_dir, set_data_dir, set_data_dir, reset_directories_to_defaults, PROJECT_ROOT
)

from bquant.data.loader import (
    load_ohlcv_data, get_data_info, load_symbol_data,
    get_available_symbols, get_available_timeframes
)

print("✅ BQuant modules imported successfully")

✅ BQuant modules imported successfully


In [11]:
reset_directories_to_defaults()

In [12]:
print(PROJECT_ROOT)
print(get_data_dir())
set_data_dir(PROJECT_ROOT/"data"/"row")
print(get_data_dir())

c:\Users\g\Documents\pro\bquant
c:\Users\g\Documents\pro\bquant\data
c:\Users\g\Documents\pro\bquant\data\row


## Шаг 1: Тестирование загрузки OANDA_XAUUSD, 60.csv

Загружаем данные OANDA формата с валидацией

In [16]:
print(get_available_symbols())


[]


In [ ]:
dfmt = load_symbol_data("XAUUSD","1h",data_source="metatrader")

DataLoadingError: Data file not found: c:\Users\g\Documents\pro\bquant\data\row\XAUUASDH1.csv (Details: file_path=c:\Users\g\Documents\pro\bquant\data\row\XAUUASDH1.csv, symbol=XAUUASD, timeframe=1h)

## Шаг 2: Тестирование загрузки XAUUSDH1.csv

Загружаем данные MetaTrader формата

In [ ]:
# Тест 2: Загрузка XAUUSDH1.csv
print("🧪 Test 2: Loading XAUUSDH1.csv")
print("=" * 35)

mt_file = get_data_dir() / "XAUUSDH1.csv"
print(f"File path: {mt_file}")
print(f"File exists: {mt_file.exists()}")

if mt_file.exists():
    try:
        # Загружаем данные MetaTrader формата
        mt_data = load_ohlcv_data(
            mt_file,
            symbol='XAUUSD',
            timeframe='1h',
            validate_data=True
        )
        
        print(f"✅ Data loaded successfully!")
        print(f"Shape: {mt_data.shape}")
        print(f"Columns: {list(mt_data.columns)}")
        print(f"Date range: {mt_data.index.min()} to {mt_data.index.max()}")
        
        print(f"\nFirst 3 rows:")
        print(mt_data.head(3))
        
        # Информация о данных
        mt_info = get_data_info(mt_data)
        print(f"\n📊 Data Info:")
        print(f"  Rows: {mt_info['rows']}")
        print(f"  Memory usage: {mt_info['memory_usage_mb']:.2f} MB")
        print(f"  Missing values: {mt_info['missing_values']}")
        
    except Exception as e:
        print(f"❌ Error loading MT data: {e}")
else:
    print("❌ MetaTrader file not found")

🧪 Test 2: Loading XAUUSDH1.csv
File path: /mnt/c/Users/user/Documents/Pro/bquant/data/row/XAUUSDH1.csv
File exists: True
❌ Error loading MT data: Failed to read CSV file: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte (Details: file_path=/mnt/c/Users/user/Documents/Pro/bquant/data/row/XAUUSDH1.csv, error='utf-8' codec can't decode byte 0xff in position 0: invalid start byte)


/mnt/c/Users/user/Documents/Pro/bquant/bquant/data/loader.py:85: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file_path, index_col=0, parse_dates=True, date_parser=date_parser)
/mnt/c/Users/user/Documents/Pro/bquant/bquant/data/loader.py:87: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(file_path, parse_dates=['time'], date_parser=date_parser)


## Шаг 3: Сравнение источников данных

Сравниваем OANDA и MetaTrader данные

In [ ]:
# Тест 3: Сравнение источников данных
print("🧪 Test 3: Comparing Data Sources")
print("=" * 35)

if 'oanda_data' in locals() and 'mt_data' in locals():
    print("📊 Data Comparison:")
    print(f"  OANDA shape: {oanda_data.shape}")
    print(f"  MetaTrader shape: {mt_data.shape}")
    
    # Сравниваем колонки
    oanda_cols = set(oanda_data.columns)
    mt_cols = set(mt_data.columns)
    common_cols = oanda_cols.intersection(mt_cols)
    
    print(f"\n📋 Columns:")
    print(f"  Common columns: {sorted(common_cols)}")
    print(f"  OANDA only: {sorted(oanda_cols - mt_cols)}")
    print(f"  MetaTrader only: {sorted(mt_cols - oanda_cols)}")
    
    # Сравниваем временные диапазоны
    if not oanda_data.index.empty and not mt_data.index.empty:
        print(f"\n📅 Time Ranges:")
        print(f"  OANDA: {oanda_data.index.min()} to {oanda_data.index.max()}")
        print(f"  MetaTrader: {mt_data.index.min()} to {mt_data.index.max()}")
        
        # Базовая статистика по ценам закрытия
        if 'close' in common_cols:
            print(f"\n💰 Close Price Stats:")
            print(f"  OANDA - Mean: {oanda_data['close'].mean():.2f}, Std: {oanda_data['close'].std():.2f}")
            print(f"  MetaTrader - Mean: {mt_data['close'].mean():.2f}, Std: {mt_data['close'].std():.2f}")
else:
    print("⚠️ Cannot compare - one or both datasets not loaded")

In [ ]:
# Настройка логирования - только WARNING+ в консоль, все остальное в файл
from bquant.core.logging_config import setup_logging
import logging

# Настраиваем логирование: WARNING+ в консоль, INFO+ в файл
logger = setup_logging(
    level="INFO",           # Общий уровень для файла
    log_to_file=True,       # Включить файловое логирование  
    log_file=project_root / "logs" / "notebook_testing.log",
    use_colors=False,       # Отключить цвета в ноутбуке
    reset_loggers=True      # Сбросить существующие настройки
)

# Устанавливаем уровень WARNING для консольного вывода
console_handler = None
for handler in logging.getLogger('bquant').handlers:
    if isinstance(handler, logging.StreamHandler) and handler.stream.name == '<stdout>':
        handler.setLevel(logging.WARNING)
        console_handler = handler
        break

print("✅ Logging configured:")
print(f"  📝 File: INFO+ → {project_root}/logs/notebook_testing.log") 
print(f"  🖥️ Console: WARNING+ only")
print(f"  📊 Console handler found: {'Yes' if console_handler else 'No'}")

## Шаг 4: Обзор доступных данных

Анализируем все доступные символы и таймфреймы

In [ ]:
# Тест 4: Обзор доступных данных
print("🧪 Test 4: Data Overview")
print("=" * 25)

# Получаем список доступных символов и таймфреймов
available_symbols = get_available_symbols(get_data_dir())
print(f"📈 Available symbols: {available_symbols}")

if available_symbols:
    print(f"\n📊 Symbol Details:")
    for symbol in available_symbols:
        timeframes = get_available_timeframes(symbol, get_data_dir())
        print(f"  {symbol}: {timeframes}")
else:
    print("❌ No symbols detected in data directory")

print(f"\n📁 Total CSV files in directory: {len(list(get_data_dir().glob('*.csv')))}")

## 🚀 Следующие шаги тестирования Data Module

### Предлагаемый план дальнейшего тестирования:

#### 📋 **Шаг 5: Data Validator (02_validator.ipynb)**
- **Валидация загруженных данных** через `validate_ohlcv_data()`
- **Проверка полноты данных** `validate_data_completeness()`  
- **Логическая согласованность цен** `validate_price_consistency()`
- **Непрерывность временных рядов** `validate_time_series_continuity()`
- **Статистические свойства** `validate_statistical_properties()`

#### 🔄 **Шаг 6: Data Processor (03_processor.ipynb)**
- **Очистка данных** `clean_ohlcv_data()` с разными параметрами
- **Удаление выбросов** `remove_price_outliers()` (z-score vs IQR)
- **Производные индикаторы** `calculate_derived_indicators()`
- **Изменение таймфрейма** `resample_ohlcv()` (1h → 4h, 1d)
- **Технические признаки** `add_technical_features()`
- **Подготовка к анализу** `prepare_data_for_analysis()`

#### 📊 **Шаг 7: Sample Data (04_samples.ipynb)** 
- **Embedded данные** `get_sample_data()` для разных форматов
- **Список датасетов** `list_datasets()` и `get_dataset_info()`
- **Валидация samples** `validate_dataset()`
- **Сравнение embedded vs реальных данных**
- **Генератор данных** `SampleDataGenerator`

#### 🔗 **Шаг 8: Integration Testing (05_integration.ipynb)**
- **Полный pipeline**: загрузка → валидация → обработка → анализ
- **Сравнение результатов** на OANDA vs MetaTrader
- **Производительность** на больших файлах
- **Edge cases** и обработка ошибок
- **Memory usage** и оптимизация

#### ⚡ **Шаг 9: Performance & Optimization (06_performance.ipynb)**
- **Бенчмарки** времени загрузки разных форматов
- **Memory profiling** больших датасетов  
- **Кэширование** и повторные загрузки
- **Параллельная обработка** нескольких файлов

### 🎯 **Критерии завершения тестирования:**
- ✅ Все функции data модуля протестированы
- ✅ Валидация проходит без критических ошибок
- ✅ Производительность в приемлемых рамках
- ✅ Edge cases обработаны корректно
- ✅ Документированы найденные проблемы